In [ ]:
# HIDDEN
import warnings
# Ignore numpy dtype warnings. These warnings are caused by an interaction
# between numpy and Cython and can be safely ignored.
# Reference: https://stackoverflow.com/a/40846742
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 9)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# HIDDEN
def df_interact(df, nrows=7, ncols=7):
    '''
    Outputs sliders that show rows and columns of df
    '''
    def peek(row=0, col=0):
        return df.iloc[row:row + nrows, col:col + ncols]

    row_arg = (0, len(df), nrows) if len(df) > nrows else fixed(0)
    col_arg = ((0, len(df.columns), ncols)
               if len(df.columns) > ncols else fixed(0))
    
    interact(peek, row=row_arg, col=col_arg)
    print('({} rows, {} columns) total'.format(df.shape[0], df.shape[1]))

def display_df(df, rows=50, cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

## Classification

### Confusion Matrices

In [ ]:
df = pd.read_csv('games.csv')
df

In [ ]:
def plot_confusion(confusion):
    sns.heatmap(confusion, annot=True, fmt='d',
                cmap="Blues", annot_kws={'fontsize': 24}, square=True,
                xticklabels=[1, 0], yticklabels=[1, 0], cbar=False)
    plt.xlabel('True')
    plt.ylabel('Predicted')

### ROC Curves

You try: Draw the ROC curve for this small dataset of 5 points. Then compute the AUC.

In [ ]:
np.random.seed(5)
X_small = X.sample(5)
y_small = y[X_small.index]
y_hat_small = clf.predict_proba(X_small)[:, 1]

In [ ]:
@interact(thres=(0, 1, 0.1))
def adjust_thres(thres=1):
    pred = (y_hat_small >= thres).astype(int)
    df = (pd.DataFrame({'y': y_small, 'y_hat': y_hat_small, 'pred': pred})
          .sort_values('y_hat'))
    fpr = df.loc[df['y'] == 0, 'pred'].mean()
    tpr = df.loc[df['y'] == 1, 'pred'].mean()
    print(f'fpr: {fpr:.2f}       tpr: {tpr:.2f}')
    display_df(df)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_small, y_hat_small)
plt.plot(fpr, tpr)
plt.scatter(fpr, tpr)
plt.xticks([0, 1/3, 2/3, 1], ['0', r'$\frac{1}{3}$', r'$\frac{2}{3}$', r'1'])
plt.yticks([0, 0.5, 1], ['0', r'$\frac{1}{2}$', r'1']);

### Logistic regression summary

The logistic or sigmoid function can be written two equivalent ways:

$$\sigma(t) = \frac{1}{1 + \exp(-t)} = \frac{\exp(t)}{1 + \exp(t)} $$

The logistic regression model assumes the following probabilities of $Y \in \{0, 1\}$ given column vector $X$:

\begin{align*}
P(Y=1|X) &= \sigma(X^T \theta) &&= \frac{1}{1 + \exp(-X^T \theta)} &= \frac{\exp(X^T\theta)}{1 + \exp(X^T\theta)} \\[10pt]
P(Y=0|X) &= \sigma(-X^T \theta) &&= \frac{1}{1 + \exp(X^T \theta)}  \\
\end{align*}

The loss most typically used to fit $\theta$ is the log loss or cross-entropy loss, which is the negative log probability of the correct (observed) $Y$ value. This loss for true $Y \in \{0, 1\}$ and predicted probability $\hat Y \in [0, 1]$ is often written:

$$-Y \log(\hat Y) - (1-Y)\log(1- \hat Y)$$

In [ ]:
def sigmoid(t):
    """The logistic or sigmoid function, denoted σ(t).
    
    Note: This is actually a special case of what is generally 
          named the "logistic" function,
          which allows for a different numerator and offset, 
          but lots of people call this the logistic function in practice.
    """
    pass

def prediction(x, theta):
    """Prediction under the logistic model for features x and parameters b."""
    pass

def squared_loss(y, y_hat):
    """Squared loss applies to any true y and predicted y_hat."""
    pass

def log_loss(y, y_hat):
    """Log loss or cross-entropy loss, assuming y is in [0, 1]."""
    pass

def empirical_risk(true_ys, predicted_ys, loss):
    """The empirical risk is the average loss for a sample."""
    pass

### Empirical Risk

Filling in $\hat Y = P(Y=1|X)$ and filling in the form of the model, we find different ways of expressing the same loss:

\begin{align*}
L(\theta) &= -Y \log(\hat Y) - (1-Y)\log(1- \hat Y) \\[10pt]
         &= - Y \log P(Y=1|X) - (1-Y) \log P(Y=0|X)  \\[10pt]
         &= - Y \log \frac{\exp(X \cdot \theta)}{1 + \exp(X \cdot \theta)} - (1-Y) \log \frac{1}{1 + \exp(X \cdot \theta)}  \\[10pt]
         &= - Y (\log(\exp(X \cdot \theta)) - \log(1 + \exp(X \cdot \theta))) - (1-Y) (-\log (1 + \exp(X \cdot \theta)))  \\[10pt]
         &= - YX \cdot \theta + Y \log(1 + \exp(X \cdot \theta))) - Y \log(1 + \exp(X \cdot \theta))) + \log (1 + \exp(X \cdot \theta))  \\[10pt]
         &= - YX \cdot \theta + \log (1 + \exp(X \cdot \theta)) \\[10pt]
         &= -\left(YX \cdot \theta + \log \sigma(-X \cdot \theta)\right)
\end{align*}

Where the last step follows from $\log (1 + \exp(X \cdot \theta)) = -(- \log (1 + \exp(X \cdot \theta))) = -\log \frac{1}{1 + \exp(X \cdot \theta)} = -\log \sigma(-X \cdot \theta)$.

The empirical risk (average loss across a sample) for a set of observations $(x_1, y_1) \dots (x_n, y_n)$ is often written:

$$R(\theta, x, y) = - \frac{1}{n} \sum_{i=1}^n \left[ y_i x_i \cdot \theta + \log \sigma(-x_i \cdot \theta) \right]$$

### Logistic Regression Gradient

Using the logistic regression model and log loss, find the gradient of the empirical risk.

First, we compute the derivative of the sigmoid function since we'll use it in our gradient calculation.

$$
\begin{aligned}
\sigma(t) &= \frac{1}{1 + e^{-t}} \\[10pt]
\sigma'(t) &= \frac{e^{-t}}{(1 + e^{-t})^2} \\[10pt]
\sigma'(t) &= \frac{1}{1 + e^{-t}} \cdot \left(1 - \frac{1}{1 + e^{-t}} \right) \\[10pt]
\sigma'(t) &= \sigma(t) (1 - \sigma(t))
\end{aligned}
$$

As a shorthand, we define $ \sigma_i = \sigma(-x_i\cdot \theta) $. We will soon need the gradient of $ \sigma_i $ with respect to the vector $ \theta $ so we will derive it now using the chain rule. 

\begin{align*}
\nabla_{\theta} \sigma_i
&= \nabla_{\theta} \sigma(-x_i\cdot \theta) \\[10pt]
&= \sigma\left(-x_i\cdot \theta\right) \left(1 - \sigma(-x_i\cdot \theta)\right)  \nabla_{\theta} \left(-x_i\cdot \theta\right) \\[10pt]
&= -\sigma_i (1 - \sigma_i) x_i 
\end{align*}

Now, we derive the gradient of the cross-entropy loss with respect to the model parameters $ \boldsymbol{\theta} $. We use the fact that $(1-\sigma_i) = \sigma(x_i \cdot \theta)$, since $\sigma(x \cdot \theta) + \sigma(-x \cdot \theta) = 1$.

\begin{align*}
R(\theta, x, y) &= - \frac{1}{n}\sum_{i=1}^n \left[ y_i x_i \cdot \theta + \log \sigma_i \right] \\[10pt]
\nabla_{\theta} R(\theta, x, y) &= - \frac{1}{n}\sum_{i=1}^n \left( y_i x_i - \frac{1}{\sigma_i} \sigma_i (1 - \sigma_i) x_i \right) \\[10pt]
                              &= - \frac{1}{n}\sum_{i=1}^n \left( y_i x_i - \sigma(x_i \cdot \theta) x_i \right) \\[10pt]
                              &= - \frac{1}{n}\sum_{i=1}^n \left(y_i - \sigma(x_i \cdot \theta)\right) x_i  \\[10pt]
\end{align*}

The gradient for a single point is the expression inside the summation. This lets us run SGD if we so desire.

In [ ]:
def gradient_descent(x, y, theta0, gradient_function, max_iter=1000000,  
                     epsilon=1e-8, lr=5, clip=1):
    """Run gradient descent on a dataset (x, y) 
    with gradient clipping and learning rate decay."""
    theta = theta0
    pass
    return theta

def risk_gradient(theta, x, y):
    """Risk gradient for a whole dataset at once."""
    pass

def logistic_regression(x, y):
    """Train a logistic regression classifier using gradient descent."""
    pass

### Applying logistic regression

In [ ]:
import sklearn.datasets
data_dict = sklearn.datasets.load_breast_cancer()
cancer = pd.DataFrame(data_dict['data'], columns=data_dict['feature_names'])
cancer['bias'] = 1.0
# Target data_dict['target'] = 0 is malignant; 1 is benign
cancer['malignant'] = 1 - data_dict['target']
cancer

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cancer, test_size=0.25, random_state=100)
print("Training Data Size: ", len(train))
print("Test Data Size: ", len(test))

In [ ]:
plt.scatter(train['mean radius'], train['malignant']);

In [ ]:
radii = np.linspace(5, 30, 50)
averages = [np.average(train[np.abs(train['mean radius']-r)<2]['malignant']) for r in radii]
plt.scatter(train['mean radius'], train['malignant']);
plt.scatter(radii, averages, color='gold');

In [ ]:
def features(t):
    pass

...

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(fit_intercept=False, C=1e9, solver='saga')
model.fit(x_train, y_train)
model.coef_

In [ ]:
plt.scatter(train['mean radius'], train['malignant']);
plt.scatter(radii, averages, color='gold');
plt.plot(radii, sigmoid(model.coef_[0,0] + radii * model.coef_[0,1]));

In [ ]:
plt.scatter(train['mean radius'], train['malignant']);
plt.scatter(radii, averages, color='gold');
plt.plot(radii, sigmoid(theta[0] + radii * theta[1]));

Checking test set performance:

Using all features:

In [ ]:
def all_features(t):
    pass

In [ ]:
def evaluate(theta, features):
    print(f'Train auc: {auc(features(train), y_train, theta):.3f}')
    print(f' Test auc: {auc(features(test), y_test, theta):.3f}')
    
evaluate(theta, all_features)